In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode
import pprint
import pandas as pd

In [2]:
client_id="8c6548155829445bb3ffb3e26cf3a501"
client_secret="15d8511764804849ba1381758b502733"

In [3]:
class SpotifyAPI(object):
    access_token=None
    access_token_expires=datetime.datetime.now()
    access_token_did_expire=True
    client_id=None
    client_secret=None
    base_url="https://accounts.spotify.com/api/token"
    
    def __init__(self,client_id,client_secret,*args,**kwargs):
        self.client_id=client_id
        self.client_secret=client_secret
    
    def get_client_credentials(self):
        client_id=self.client_id
        client_secret=self.client_secret
        if client_secret == None or client_id == None:
            raise Exception (" Please set client_id and client_secret")
            
        "returns as base64"
        client_creds=f"{client_id}:{client_secret}"
        client_creds_b64=base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
        
    def get_token_header(self):
        client_creds_b64=self.get_client_credentials()
        return {"Authorization":f"Basic {client_creds_b64}"}
    
    def get_token_data(self):
        return {"grant_type":"client_credentials"}
    
    def perform_auth(self):
        base_url=self.base_url
        token_data=self.get_token_data()
        token_header=self.get_token_header()
        r=requests.post(base_url,data=token_data,headers=token_header)
        
        if r.status_code not in range(200,299):
            return False
        token_response_data=r.json()
        now=datetime.datetime.now()
        access_token=token_response_data["access_token"]
        self.access_token=access_token
        expires_in = token_response_data["expires_in"] #seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        did_expire = expires < now
        return True

In [4]:
spotify=SpotifyAPI(client_id,client_secret)

In [5]:
spotify.perform_auth()

True

In [6]:
access_token=spotify.access_token
access_token

'BQD66EaEI4w82YNZrHW8ApamSO80D8pIB4BbrMJxHY_hxI1GquQH_sABaK5RqKep5Qqe4qtwVNp7z3_1Ttc'

In [7]:
artists=["Arijit Singh","Tanishk Bagchi","Neha Kakkar","Badshah","Shreya ghoshal"]
artist_details=pd.DataFrame(columns=["Name","id","Popularity","Followers","Genres"])

for i in artists:
    endpoint="https://api.spotify.com/v1/search"
    headers={"Authorization":f"Bearer {access_token}"}
    data=urlencode({"q":i,"type":"artist","market":"IN","offset":0})
    lookup_url=f"{endpoint}?{data}"
    r=requests.get(lookup_url,headers=headers)
    result=r.json()
    output=result['artists']["items"][0]
    position=len(artist_details)
    artist_details.loc[position]=[output["name"],output["id"],output["popularity"],output["followers"]["total"],output["genres"]]

artist_details

,Name,id,Popularity,Followers,Genres
0,Arijit Singh,4YRxDV8wJFPHPTeXepOstw,83,15618858,"[desi pop, filmi, modern bollywood]"
1,Tanishk Bagchi,4f7KfxeHq9BiylGmyXepGt,76,331471,"[desi hip hop, desi pop, filmi, modern bollywood]"
2,Neha Kakkar,5f4QpKfy7ptCHwTqspnSJI,76,10637030,"[desi hip hop, desi pop, filmi, modern bollywood]"
3,Badshah,0y59o4v8uw5crbN9M3JiL1,75,7389686,"[desi hip hop, desi pop, haryanvi pop, modern ..."
4,Shreya Ghoshal,0oOet2f43PA68X5RxKobEy,74,4440538,"[desi hip hop, desi pop, filmi, modern bollywo..."


In [8]:
def every_artist(id):
    albums_url=f"https://api.spotify.com/v1/artists/{id}/albums"
    headers={"Authorization":f"Bearer {access_token}"}
    data=urlencode({"include_groups":"album,single","market":"IN","limit":50}) #50 is the max limit
    lookup_url=f"{albums_url}?{data}"
    r=requests.get(lookup_url,headers=headers)
    if r.status_code in range(200,299):
        album_details=r.json()["items"]
        return album_details
    return False


In [9]:
def every_track(album_id):
    endpoint=f"https://api.spotify.com/v1/albums/{album_id}/tracks"
    headers={"Authorization":f"Bearer {access_token}"}
    data=urlencode({"market":"IN","limit":50}) #50 is the max limit
    lookup_url=f"{endpoint}?{data}"
    r=requests.get(lookup_url,headers=headers)
    if r.status_code in range(200,299):
        track_details=r.json()["items"]
        return track_details
    return False

In [10]:
def each_track_features(track_id,track_features):
    track_endpoint=f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers={"Authorization":f"Bearer {access_token}"}
    r=requests.get(track_endpoint,headers=headers)
    if r.status_code in range(200,299):
        data=r.json()
        cols=['danceability','energy','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
        for col in cols:
            track_features.append(data[col])
        return track_features
    return False

In [11]:
all_cols=["Artist ID","Artist name","Album ID","Album name","Album release date","Track ID","Track name",'danceability','energy','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
cumilated=pd.DataFrame(columns=all_cols)

In [12]:
count=0
for name,artist_id in zip(artist_details["Name"],artist_details["id"]):
    album_details=every_artist(artist_id)
    for j in range(len(album_details)):
        album=album_details[j]
        album_id=album["id"]
        track_details=every_track(album_id)
        for k in range(len(track_details)):
            count=count+1
            track_features=[]
            track=track_details[k]
            track_id=track["id"]
            track_features=[artist_id,name,album["id"],album["name"],album["release_date"],track["id"],track["name"]]
            features=each_track_features(track_id,track_features) 
            position=len(cumilated)
            cumilated.loc[position]=features
count            

855

In [13]:
cumilated.to_csv("artists_indian.csv", sep = ',')